In [185]:
import pandas as pd
import numpy as np
import datetime
import sys
import codecs
import re
import urllib3
import itertools, collections
 
import nltk  # Natural Language Processing
#nltk.download('punkt')
#nltk.download('all')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords # list of words
from collections import Counter  # optimized way to do this
import string  # list(string.punctuation) - produces a list of punctuations
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import RegexpTokenizer

import copy
from itertools import product, tee, combinations, chain
from nltk.stem import PorterStemmer
from operator import itemgetter # help with dataframes
 
from scipy.spatial.distance import cosine
 
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle
 


In [206]:
# originaldf = pd.read_csv('./twitterBios.csv')
# UserID = originaldf.index
# originaldf['UserID'] = UserID
# originaldf = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# df = originaldf
# df.head()
org_df = pd.read_csv('./twitterBios.csv')
org_df = org_df.loc[:, ~org_df.columns.str.contains('^Unnamed')]
UserID = org_df.index
org_df['UserID'] = UserID
df = org_df
df.head()

,user,bio,UserID
0,MikePaineShow,#voiceover talent voice actor & radio guy for ...,0
1,gablescinema,South Florida's premiere nonprofit cinema for ...,1
2,dhumpachika,Follow us for the best of what's happening abo...,2
3,DavyVara,Filmmaker Writer Activist Aspiring Actor... I ...,3
4,keeeykeh,Dedicated student of the beautiful game. Head ...,4


In [207]:
org_df.head()

,user,bio,UserID
0,MikePaineShow,#voiceover talent voice actor & radio guy for ...,0
1,gablescinema,South Florida's premiere nonprofit cinema for ...,1
2,dhumpachika,Follow us for the best of what's happening abo...,2
3,DavyVara,Filmmaker Writer Activist Aspiring Actor... I ...,3
4,keeeykeh,Dedicated student of the beautiful game. Head ...,4


In [208]:
# some constants for regex
START_OF_LINE = r"^"
OPTIONAL = "?"
ANYTHING = "."
ZERO_OR_MORE = "*"
ONE_OR_MORE = "+"

SPACE = "\s"
SPACES = SPACE + ONE_OR_MORE
NOT_SPACE = "[^\s]" + ONE_OR_MORE
EVERYTHING_OR_NOTHING = ANYTHING + ZERO_OR_MORE

ERASE = ""
FORWARD_SLASH = "\/"
NEWLINES = r"[\r\n]"

In [209]:
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

def clean_up(text):
    RE_TWEET = START_OF_LINE + "RT" + SPACES
    text = re.sub(RE_TWEET, ERASE, text)
    HYPERLINKS = ("http" + "s" + OPTIONAL + ":" + FORWARD_SLASH + FORWARD_SLASH
              + NOT_SPACE + NEWLINES + ZERO_OR_MORE)

    text = re.sub(HYPERLINKS, ERASE, text)
    HASH = "#"
    text = re.sub(HASH, ERASE, text)
    return text

In [210]:
df['bio'] = df['bio'].apply(clean_up)

In [211]:
df

,user,bio,UserID
0,MikePaineShow,voiceover talent voice actor & radio guy for 3...,0
1,gablescinema,South Florida's premiere nonprofit cinema for ...,1
2,dhumpachika,Follow us for the best of what's happening abo...,2
3,DavyVara,Filmmaker Writer Activist Aspiring Actor... I ...,3
4,keeeykeh,Dedicated student of the beautiful game. Head ...,4
...,...,...,...
180,staughumane,The St. Augustine Humane Society offers lifesa...,195
181,soccerjill,Designer/maker of 👜😷 & accessories who doubles...,196
182,CRHM_Dickson,State Historic Site. Education. Events. Tours....,197
183,QCHSPantherPack,Quakertown Students Telling/Tweeting their Sto...,198


In [212]:
def remove_punct(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct
df['bio'] = df['bio'].apply(lambda x: remove_punct(x))

In [213]:
tokenizer = RegexpTokenizer(r'\w+')
df['bio'] = df['bio'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [214]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
df['bio'] = df['bio'].apply(lambda x:  remove_stopwords(x))

In [215]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

df['bio'] = df['bio'].apply(lambda x: word_lemmatizer(x))


In [216]:
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

df['bio'] = df['bio'].apply(lambda x: word_stemmer(x))

In [217]:
df

,user,bio,UserID
0,MikePaineShow,voiceov talent voic actor radio guy 30 year wi...,0
1,gablescinema,south florida premier nonprofit cinema best in...,1
2,dhumpachika,follow u best what happen cycl bike around wor...,2
3,DavyVara,filmmak writer activist aspir actor make film ...,3
4,keeeykeh,dedic student beauti game head coach colleg de...,4
...,...,...,...
180,staughumane,st augustin human societi offer lifesav resour...,195
181,soccerjill,designermak accessori doubl paraleg soccer pla...,196
182,CRHM_Dickson,state histor site educ event tour venu rental ...,197
183,QCHSPantherPack,quakertown student tellingtweet stori quakerto...,198


In [218]:
#TF-IDF vectorizer
tfv = TfidfVectorizer(stop_words = "english",ngram_range = (1,1))
#transform
vec_text = tfv.fit_transform(df.bio)
#returns a list of words.
words = tfv.get_feature_names()
features = tfv.transform(df.bio.values)

In [219]:
#setup kmeans clustering
n_clst = 6
kmeans = KMeans(n_clusters = n_clst, n_init = 17, tol = 0.01, max_iter = 200)
#fit the data 
kmeans.fit(vec_text)
#this loop transforms the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

yhat = kmeans.predict(features)
df['BioClusters'] = kmeans.labels_
df

0 : commun, dont, child, follow, provid, drive, support, busi, student, market
1 : love, god, dm, im, friend, bf, kabir, beach, life, wife
2 : writer, ask, make, cojon, think, question, aspir, filmmak, actor, peopl
3 : brooklyn, aka, area, villag, old, south, jpstrategiesco, brownunivers, cofound, coblackart
4 : school, mom, high, coach, district, wife, middl, girl, boy, father
5 : educ, scienc, nonprofit, natur, engag, advanc, technolog, world, read, provid


,user,bio,UserID,BioClusters
0,MikePaineShow,voiceov talent voic actor radio guy 30 year wi...,0,1
1,gablescinema,south florida premier nonprofit cinema best in...,1,5
2,dhumpachika,follow u best what happen cycl bike around wor...,2,5
3,DavyVara,filmmak writer activist aspir actor make film ...,3,2
4,keeeykeh,dedic student beauti game head coach colleg de...,4,4
...,...,...,...,...
180,staughumane,st augustin human societi offer lifesav resour...,195,1
181,soccerjill,designermak accessori doubl paraleg soccer pla...,196,4
182,CRHM_Dickson,state histor site educ event tour venu rental ...,197,5
183,QCHSPantherPack,quakertown student tellingtweet stori quakerto...,198,4


In [220]:
for i in range(n_clst):
    Clust1wBios=df[df['BioClusters']==i]
    Clust1wBios.to_csv('{}/{}.csv'.format('ALGO2_BioClusters',i))

In [224]:
df.drop(['bio'], axis=1)
finalS1 = pd.merge(org_df,df, on='UserID')
finalS1.head()

,user_x,bio_x,UserID,user_y,bio_y,BioClusters
0,MikePaineShow,#voiceover talent voice actor & radio guy for ...,0,MikePaineShow,voiceov talent voic actor radio guy 30 year wi...,1
1,gablescinema,South Florida's premiere nonprofit cinema for ...,1,gablescinema,south florida premier nonprofit cinema best in...,5
2,dhumpachika,Follow us for the best of what's happening abo...,2,dhumpachika,follow u best what happen cycl bike around wor...,5
3,DavyVara,Filmmaker Writer Activist Aspiring Actor... I ...,3,DavyVara,filmmak writer activist aspir actor make film ...,2
4,keeeykeh,Dedicated student of the beautiful game. Head ...,4,keeeykeh,dedic student beauti game head coach colleg de...,4


In [225]:
for i in range(n_clst):
    Clust1wBios=finalS1[finalS1['BioClusters']==i]
    Clust1wBios.to_csv('{}/{}.csv'.format('ALGO2_BioClusters/clean',i))